In [18]:
# Imports 
import string
import pandas as pd
import numpy as np
import time
from scipy.stats.stats import pearsonr 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import NearestNeighbors

In [2]:
# load the data, keep only selected columns
# Read the file using super fast Pandas.read_csv
def load_required_data(path, required_columns):
    dataframe = pd.read_json(path, lines=True)
    dataframe = dataframe[required_columns]
    return dataframe, dataframe.values

In [3]:
dataframe, values = load_required_data('../train.json', ["asin", "reviewerID", "overall"])

In [4]:
dataframe[:10]

,asin,reviewerID,overall
0,B00AMR1HZ8,A00100742Q4O8VH0YMUBZ,4
1,B006C1ZSO4,A00100742Q4O8VH0YMUBZ,5
2,B00ANT8OF6,A00100742Q4O8VH0YMUBZ,4
3,B008LY1B32,A00100742Q4O8VH0YMUBZ,4
4,B0080JJLBW,A00100742Q4O8VH0YMUBZ,5
5,B007KPT2N4,A00100742Q4O8VH0YMUBZ,1
6,B00EQ2TYMS,A00100742Q4O8VH0YMUBZ,4
7,B007PW1BRC,A00100742Q4O8VH0YMUBZ,2
8,B009P8EMCK,A00100742Q4O8VH0YMUBZ,5
9,B006YUVTK0,A00100742Q4O8VH0YMUBZ,5


In [5]:
dataframe.shape

(665666, 3)

In [6]:
# Create user-item matrix given a matrix and mapping from original matrix columns to new matrix rows/columns
def create_user_item_matrix(data, rowMapping, columnMapping):
    # Create table of users to items by taking row and column pivots
    # Rows of user-item table are users
    rows, row_pos = np.unique(data[:, rowMapping], return_inverse=True)
    # Columns of user-item table are movies
    cols, col_pos = np.unique(data[:, columnMapping], return_inverse=True)

    pivot_table = np.zeros((len(rows), len(cols)), dtype=data.dtype)
    pivot_table[row_pos, col_pos] = data[:, 2]
    return pivot_table, rows, cols

In [7]:
ratings_matrix, rows, cols = create_user_item_matrix(values, rowMapping=1, columnMapping=0)
print (ratings_matrix.shape)

(87271, 13209)


In [8]:
pd.DataFrame(ratings_matrix[:10], index = rows[:10], columns= cols)

,B004A9SDD8,B004AFQAUA,B004AHBBPW,B004ALVL6W,B004AMAIZQ,B004AMDC86,B004ANC00Q,B004ANE2WU,B004ANMWPY,B004AZH4C8,...,B00LDNE3FG,B00LDNQHJ6,B00LEIT6K2,B00LFLOEYG,B00LIAMYCI,B00LITLOMK,B00LMLW8T2,B00LMLXTJK,B00LP1MVSW,B00LUEMK44
A00100742Q4O8VH0YMUBZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A001619027H9L9EG4UVRB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A002359833QJM7OQHCXWY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0037670NPLI11RBWYFA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A003841815JTX0JFLR8B1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0038872349TB5N0JHQQW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00455683H6M1GQZMPQPV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0047670XAAJD587LXS7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00501041JRAPWYLPQ4TE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00507662MEMHI1YMGQ15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
def get_index_map(arr):
    index_map = {}
    for el in arr:
        index_map[el] = len(index_map)
    return index_map

In [10]:
def pearson_coeff(target, others):
    return np.corrcoef(matrix[target], matrix[others])[0][1:]

In [11]:
# Get unique movies and users
unique_items = np.unique(values[:,0])
unique_users = np.unique(values[:,1])

In [16]:
unique_items.shape

(13209,)

In [17]:
unique_users.shape

(87271,)

In [12]:
# Create mapping from unique list to 
item_idx_map = get_index_map(unique_items)
user_idx_map = get_index_map(unique_users)

In [13]:
mean_ratings = np.true_divide(ratings_matrix.sum(1), (ratings_matrix!=0).sum(1))

In [14]:
K = 0.1

In [15]:
def get_users_who_rated_item(item_id):
    return np.where(ratings_matrix[:,item_idx_map[item_id]] > 0)[0]

def get_users_who_rated_item_by_index(item_idx):
    return np.where(ratings_matrix[:,item_idx] > 0)

def predict(user_id, item_id):
    item_idx = item_idx_map[item_id]
    user_idx = user_idx_map[user_id]
    other_users_idx = get_users_who_rated_item(item_id)
    if user_idx in other_users_idx:
        np.delete(other_users_idx, user_idx)
    pearson_correlation = [pearsonr(other_user, ratings_matrix[user_idx])[0] for other_user in ratings_matrix[other_users_idx]]
    mean_others = mean_ratings[other_users_idx]
    rating_others = ratings_matrix[other_users_idx,item_idx]
    prediction = mean_ratings[user_idx] + K * np.sum(pearson_correlation * (rating_others - mean_others))
    return prediction

In [16]:
predict(u'A0698182HBSA2D2MEX40', 'B004K44R74')

3.3358391387334074

In [17]:
np.mean(mean_ratings)

4.007764456541003

In [26]:
test_dataframe, test_values = load_required_data('../test.json', ["asin", "reviewerID", "overall"])

In [27]:
test_dataframe['reviewerID'].unique().shape

(87271,)

In [34]:
# Returns DF containing predictions, MAE, RMSE
def get_predictions(dataframe):
    df = dataframe.copy()
    print ("Predicting ratings..")
    start_time = time.time()
    df['Prediction'] = df.apply(lambda row: predict_row(row), axis=1)
    MAE = round(mean_absolute_error(df['overall'], df['Prediction']), 2)
    RMSE = round(np.sqrt(mean_squared_error(df['overall'], df['Prediction'])), 2)
    print "Done Predicting in",  round((time.time() - start_time), 2), "seconds"
    return df, MAE, RMSE

def predict_row(row):
    prediction = predict(row['reviewerID'], row['asin'])
    print ("Actual = ", row['overall'], "Predictioned = ", prediction)
    return prediction

In [55]:
sparse_ratings = csr_matrix(ratings_matrix, dtype='float')

In [56]:
sparse_ratings

<87271x13209 sparse matrix of type '<type 'numpy.float64'>'
	with 665666 stored elements in Compressed Sparse Row format>

In [57]:
model_knn = NearestNeighbors(metric = 'cosine', n_neighbors = 5)
model_knn.fit(sparse_ratings)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [58]:
distances,indices = model_knn.kneighbors(ratings_matrix[0].reshape(1, -1), n_neighbors=6)

In [111]:
probable_items = list()
for user in indices[0]:
    if user == 0:
        print "Recommendations for user ", unique_users[0]
        continue
    # all other users
    for items in np.nonzero(ratings_matrix[user]):
        for item in items:
            if ratings_matrix[0][item] == 0:
                probable_items.append(unique_items[item])

predictions = {}
for item in probable_items:
    predictions[item] = predict(unique_users[0], item)

Recommendations for user  A00100742Q4O8VH0YMUBZ


In [112]:
predictions

{u'B004SJ3204': 4.106218533325619,
 u'B004V5QWU2': 3.93051163199979,
 u'B0069Y4E08': 4.00579506630468,
 u'B008JYBDYG': 3.9942586916326808,
 u'B008L18QVU': 4.03342585113906,
 u'B008YHXCOM': 3.994893779010338,
 u'B009PN5UF8': 3.8666416200915057,
 u'B009S984J8': 4.086594276696867,
 u'B00A2XU9IS': 3.9967451756053984,
 u'B00A8ECU24': 3.9469270003819887,
 u'B00AFZKPU0': 3.978001160099666,
 u'B00AM5JTV4': 3.9621466737191025,
 u'B00D9CNAOY': 3.97986068524672,
 u'B00DUGCLY4': 3.9748173766410417,
 u'B00EAPT2NM': 3.9999175345935742,
 u'B00ECYWVQG': 4.045165914041073,
 u'B00ED0R0EW': 3.999747343795896,
 u'B00EW3STPY': 3.94785334794105,
 u'B00GPQH1MS': 3.925950200806318,
 u'B00H5CXI6Y': 3.9812485482579967}